## Tutorial 3: Fuzzy matching and bucketing across events using UMAP and hypergraphs

Part of the [Python security RAPIDS GPU & graph one-liners](Tutorial_0_Intro.ipynb)

All GPU Python data science tutorials: [RAPIDS Academy github](https://github.com/RAPIDSAcademy/rapidsacademy)


We often need to map out events, such as for bucketing events into incidents, or finding coordinated bot activity. This can be tough when there are many events, and the correlations are on fuzzy values such as time stamps.

This tutorial is inspired by by Matt Swann (Microsoft)'s GraphThePlanet talk on [Intrusion detection with graphs](https://www.slideshare.net/MattSwann1). The original recorded demo used the technique for mapping COVID misinformation bot events on Twitter. We redo it with Project Mordor elastic search APT simulation data.

The analysis walks through:

* Fuzzy match: UMAP nearest-neighbor by generically comparing combinations of many event attributes, e.g., rough time, bytes, and specific username  (`cuml`'s `umap`)
* Exact match: Link on a few individual attributes, such as username (`cudf`, `graphistry`)
* Visualize (`graphistry`)



### Setup

To setup and test your RAPIDS environment (cudf, blazingsql, graphistry, ...), use the [setup guide](setup.ipynb).

### Get data

In [1]:
! wget https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/small/windows/privilege_escalation/empire_invoke_runas.tar.gz \
    && ls -alh empire_invoke_runas.tar.gz \
    && tar -xvf empire_invoke_runas.tar.gz \
    && ls -alh empire_invoke_runas_2019-05-18204300.json

--2020-05-27 21:25:47--  https://raw.githubusercontent.com/hunters-forge/mordor/master/datasets/small/windows/privilege_escalation/empire_invoke_runas.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 690886 (675K) [application/octet-stream]
Saving to: ‘empire_invoke_runas.tar.gz’

empire_invoke_runas 100%[===================>] 674.69K  --.-KB/s    in 0.01s   

2020-05-27 21:25:47 (65.0 MB/s) - ‘empire_invoke_runas.tar.gz’ saved [690886/690886]

-rw-r--r-- 1 graphistry graphistry 675K May 27 21:25 empire_invoke_runas.tar.gz
empire_invoke_runas_2019-05-18204300.json
-rw-r--r-- 1 graphistry graphistry 13M May 19  2019 empire_invoke_runas_2019-05-18204300.json


In [6]:
#! rm -f empire_invoke_runas.tar.gz && rm -f empire_invoke_runas_2019-05-18204300.json

In [2]:
! head -n 3 empire_invoke_runas_2019-05-18204300.json

{"@timestamp":"2019-05-18T20:42:59.034Z","@metadata":{"beat":"winlogbeat","type":"doc","version":"6.7.0","topic":"winlogbeat"},"task":"Pipeline Execution Details","level":"Information","keywords":["Classic"],"beat":{"version":"6.7.0","name":"WECserver","hostname":"WECserver"},"host":{"name":"WECserver"},"event_id":800,"record_number":"101776","message":"Pipeline execution details for command line:             Start-Sleep -Seconds $sleepTime;\n. \n\nContext Information: \n\tDetailSequence=1\n\tDetailTotal=1\n\n\tSequenceNumber=70994\n\n\tUserId=SHIRE\\nmartha\n\tHostName=ConsoleHost\n\tHostVersion=5.1.17763.316\n\tHostId=76b548b3-2221-41d9-8e53-ad5dff2c5c4e\n\tHostApplication=C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe -noP -sta -w 1 -enc SQBmACgAJABQAFMAVgBFAHIAUwBpAE8AbgBUAEEAQgBsAEUALgBQAFMAVgBlAHIAcwBJAE8ATgAuAE0AQQBqAG8AcgAgAC0AZwBFACAAMwApAHsAJAAyADQAMQBmAD0AWwByAEUAZgBdAC4AQQBzAHMAZQBtAEIATAB5AC4ARwBlAHQAVABZAHAARQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAb

## Load data

Note that the RAPIDS json logs reader is *not* yet GPU-accelerated, so we use pandas. We recommend using accelerated formats like parquet or csv when possible.

In [3]:
import cudf, cuml, graphistry, json, pandas as pd
import logging
logger = logging.getLogger()

from pandas.api.types import is_datetime64_any_dtype

In [4]:
pd.set_option('display.max_columns', 100)
logger.setLevel(logging.INFO) #DEBUG}

In [5]:
%%time

raw_df = pd.read_json('./empire_invoke_runas_2019-05-18204300.json', 
                    lines=True)
print('shape', raw_df.shape)
print('columns', raw_df.columns)

raw_df.sample(3)

shape (2581, 23)
columns Index(['@metadata', '@timestamp', 'activity_id', 'beat', 'computer_name',
       'event_data', 'event_id', 'host', 'keywords', 'level', 'log_name',
       'message', 'opcode', 'process_id', 'provider_guid', 'record_number',
       'source_name', 'task', 'thread_id', 'type', 'user', 'user_data',
       'version'],
      dtype='object')
CPU times: user 121 ms, sys: 42.9 ms, total: 164 ms
Wall time: 165 ms


,@metadata,@timestamp,activity_id,beat,computer_name,event_data,event_id,host,keywords,level,log_name,message,opcode,process_id,provider_guid,record_number,source_name,task,thread_id,type,user,user_data,version
643,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18T20:43:37.928Z,NaN,"{'name': 'WECserver', 'hostname': 'WECserver',...",HR001.shire.com,"{'Signature': 'Microsoft Windows', 'SignatureS...",7,{'name': 'WECserver'},NaN,Information,Microsoft-Windows-Sysmon/Operational,Image loaded:\nRuleName: \nUtcTime: 2019-05-18...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2238060,Microsoft-Windows-Sysmon,Image loaded (rule: ImageLoad),3144.0,wineventlog,"{'identifier': 'S-1-5-18', 'name': 'SYSTEM', '...",NaN,3.0
997,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18T20:43:41.297Z,NaN,"{'version': '6.7.0', 'name': 'WECserver', 'hos...",HR001.shire.com,{'Image': 'C:\Windows\System32\WindowsPowerShe...,12,{'name': 'WECserver'},NaN,Information,Microsoft-Windows-Sysmon/Operational,Registry object added or deleted:\nRuleName: \...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2238304,Microsoft-Windows-Sysmon,Registry object added or deleted (rule: Regist...,3144.0,wineventlog,"{'identifier': 'S-1-5-18', 'name': 'SYSTEM', '...",NaN,2.0
432,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18T20:43:35.802Z,NaN,"{'hostname': 'WECserver', 'version': '6.7.0', ...",HR001.shire.com,{'ImageLoaded': 'C:\Windows\System32\shlwapi.d...,7,{'name': 'WECserver'},NaN,Information,Microsoft-Windows-Sysmon/Operational,Image loaded:\nRuleName: \nUtcTime: 2019-05-18...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2237878,Microsoft-Windows-Sysmon,Image loaded (rule: ImageLoad),3144.0,wineventlog,"{'identifier': 'S-1-5-18', 'name': 'SYSTEM', '...",NaN,3.0


## Flatten & type

Need each column to be a primitive type like a time, str, or int.

* Flatten every json col into multiple primitive cols
* Time str -> time


In [6]:
import ast
def str_to_json(s):
    if pd.isna(s):
        return {}
    try:
        return ast.literal_eval(str(s))
    except:
        #logger.error('oops', exc_info=True)
        #logger.error('str: %s', s)
        return {}
    
def flatten_json_str_cols(df, cols):
    out_df = df.copy(deep=False)
    for col in cols:
        flattened_df = pd.io.json.json_normalize(df[col].apply(str_to_json))
        #out_df.drop(labels=[col], inplace=True)
        logger.debug('%s -> %s', col, flattened_df.columns)        
        for c in flattened_df.columns:
            #print('adding', col, c)
            out_df[f'{col}_{c}'] = flattened_df[c]
    return out_df

In [7]:
df = raw_df.copy(deep=False)
df['@timestamp'] = pd.to_datetime(df['@timestamp'])
df = flatten_json_str_cols(df, ['event_data', 'host', 'user'])

df['event_data_UtcTime'] = pd.to_datetime(df['event_data_UtcTime'])

print(raw_df.shape, '->', df.shape)

df.sample(3)

(2581, 23) -> (2581, 165)


,@metadata,@timestamp,activity_id,beat,computer_name,event_data,event_id,host,keywords,level,log_name,message,opcode,process_id,provider_guid,record_number,source_name,task,thread_id,type,user,user_data,version,event_data_AccessList,event_data_AccessMask,event_data_AccessReason,event_data_Application,event_data_AuthenticationPackageName,event_data_CallTrace,event_data_CommandLine,event_data_Company,event_data_ContextInfo,event_data_CreationUtcTime,event_data_CurrentDirectory,event_data_Description,event_data_DestAddress,event_data_DestPort,event_data_DestinationHostname,event_data_DestinationIp,event_data_DestinationIsIpv6,event_data_DestinationPort,event_data_DestinationPortName,event_data_Details,event_data_Device,event_data_Direction,event_data_DisabledPrivilegeList,event_data_ElevatedToken,event_data_EnabledPrivilegeList,event_data_EventCountTotal,event_data_EventIdx,...,event_data_SourceHandleId,event_data_SourceHostname,event_data_SourceImage,event_data_SourceIp,event_data_SourceIsIpv6,event_data_SourcePort,event_data_SourcePortName,event_data_SourceProcessGUID,event_data_SourceProcessId,event_data_SourceThreadId,event_data_Status,event_data_SubjectDomainName,event_data_SubjectLogonId,event_data_SubjectUserName,event_data_SubjectUserSid,event_data_TargetDomainName,event_data_TargetFilename,event_data_TargetHandleId,event_data_TargetImage,event_data_TargetInfo,event_data_TargetLinkedLogonId,event_data_TargetLogonGuid,event_data_TargetLogonId,event_data_TargetObject,event_data_TargetOutboundDomainName,event_data_TargetOutboundUserName,event_data_TargetProcessGUID,event_data_TargetProcessId,event_data_TargetServerName,event_data_TargetSid,event_data_TargetUserName,event_data_TargetUserSid,event_data_TerminalSessionId,event_data_TicketEncryptionType,event_data_TicketOptions,event_data_TokenElevationType,event_data_TransactionId,event_data_TransmittedServices,event_data_User,event_data_UtcTime,event_data_VirtualAccount,event_data_WorkstationName,event_data_param1,event_data_param2,event_data_param3,host_name,user_domain,user_identifier,user_name,user_type
1817,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:44:02.441000+00:00,NaN,"{'name': 'WECserver', 'hostname': 'WECserver',...",HR001.shire.com,{'param2': ' DetailSequence=1 DetailTotal=1 ...,800,{'name': 'WECserver'},[Classic],Information,Windows PowerShell,Pipeline execution details for command line: ...,Info,NaN,NaN,102085,PowerShell,Pipeline Execution Details,NaN,wineventlog,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,$taskURI = $sc...,\tDetailSequence=1\n\tDetailTotal=1\n\n\tSeque...,"CommandInvocation(Get-Random): ""Get-Random""\nP...",WECserver,NaN,NaN,NaN,NaN
720,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:43:38.118000+00:00,NaN,"{'name': 'WECserver', 'hostname': 'WECserver',...",HR001.shire.com,{'ProcessGuid': '{03ba39f5-6e79-5ce0-0000-0010...,7,{'name': 'WECserver'},NaN,Information,Microsoft-Windows-Sysmon/Operational,Image loaded:\nRuleName: \nUtcTime: 2019-05-18...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2238118,Microsoft-Windows-Sysmon,Image loaded (rule: ImageLoad),3144.0,wineventlog,"{'name': 'SYSTEM', 'domain': 'NT AUTHORITY', '...",NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Corporation,NaN,NaN,NaN,ATL Module for Windows XP (Unicode),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-18 20:43:38.081,NaN,NaN,NaN,NaN,NaN,WECserver,NT AUTHORITY,S-1-5-18,SYSTEM,User
261,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:43:28.676000+00:00,{c93a7728-017f-0000-7108-4aca7f01d501},"

In [8]:
{x: len(df[x].astype(str).unique()) for x in df.columns}

{'@metadata': 1,
 '@timestamp': 1213,
 'activity_id': 330,
 'beat': 3,
 'computer_name': 3,
 'event_data': 2442,
 'event_id': 43,
 'host': 1,
 'keywords': 4,
 'level': 4,
 'log_name': 5,
 'message': 2124,
 'opcode': 6,
 'process_id': 13,
 'provider_guid': 8,
 'record_number': 2581,
 'source_name': 5,
 'task': 37,
 'thread_id': 34,
 'type': 1,
 'user': 16,
 'user_data': 43,
 'version': 5,
 'event_data_AccessList': 5,
 'event_data_AccessMask': 5,
 'event_data_AccessReason': 4,
 'event_data_Application': 8,
 'event_data_AuthenticationPackageName': 3,
 'event_data_CallTrace': 48,
 'event_data_CommandLine': 6,
 'event_data_Company': 2,
 'event_data_ContextInfo': 326,
 'event_data_CreationUtcTime': 7,
 'event_data_CurrentDirectory': 4,
 'event_data_Description': 132,
 'event_data_DestAddress': 7,
 'event_data_DestPort': 8,
 'event_data_DestinationHostname': 5,
 'event_data_DestinationIp': 8,
 'event_data_DestinationIsIpv6': 3,
 'event_data_DestinationPort': 19,
 'event_data_DestinationPortNa

## Encode columns as values between 0 and 1

* Turns smaller string columns into multiple 0/1 indicator columns ("is_x", "is_y", ...) otherwise drops
* Good to play with

In [9]:
#pdf -> gdf['id', col_0:(0,1), ...]
def quick_encode(df, cols=None, onehot_threshold=None, id_col=None,  normalize=True):
    
        
    if onehot_threshold is None:
        onehot_threshold = min(round(0.1 * len(df)), 100)
        logger.debug('threshold on %s -> %s', len(df), onehot_threshold)
    
    if cols is None:
        cols = df.columns
        
    df_primitive = df.copy(deep=False)
    for col in df.columns:
        if df[col].dtype.name == 'object':
            df[col] = df[col].astype(str)
    
    gdf = cudf.from_pandas(df)
    
    out_gdf = gdf.copy(deep=False)
    if id_col is None:
        out_gdf = out_gdf[[]].reset_index().rename(columns={'index': 'id'}, copy=False)
    else:
        out_gdf = out_gdf[[]]
        out_gdf['id'] = gdf[id_col]
        
    #logger.debug('starting cols: %s', out_gdf.columns)
           
    for col in cols:
        print('col', col)
        if col != id_col and col != 'id':
            unique = gdf[col].unique()
            if len(unique) < 2:
                logger.debug('%s boring', col)
                continue
            elif gdf[col].dtype.name == 'object':
                if len(unique) < onehot_threshold:
                    logger.debug('%s onehot', col)
                    out_gdf[col] = gdf[col]
                    out_gdf = out_gdf.one_hot_encoding(column=col, prefix='onehot_%s' % col, cats=unique)
                    out_gdf.drop(labels=[col], inplace=True)
                else:
                    logger.debug('%s str -> many uniques; dropping %s', col, len(unique))
                    continue
            elif is_datetime64_any_dtype(df[col]):
                logger.debug('%s date->int64', col)
                out_gdf[col] = gdf[col].astype('int64')
            else:
                logger.debug('%s numeric, preserve %s', col, gdf[col].dtype.name)
                out_gdf[col] = gdf[col]
                continue
        else:
            logger.debug('%s skip id col', col)
            continue
    
    if normalize:
        for col in out_gdf.columns:
            if col != 'id_col' and col != 'id':
                ok = False
                out_gdf[col] = out_gdf[col].fillna(0.0)
                try:
                    out_gdf[col] = out_gdf[col].scale()
                    ok=True
                except:
                    if len(out_gdf[col].unique()) > 0:
                        mx = out_gdf[col].max()
                        mn = out_gdf[col].min()
                        w = mx - mn
                        if w > 0.0:
                            out_gdf[col] = out_gdf[col] - out_gdf[col].min()
                            out_gdf[col] = out_gdf[col] / w
                            ok = True
                if not ok:
                    out_gdf.drop(labels=[col], inplace=True)
            
    return out_gdf

df_encoded = quick_encode(df).to_pandas()
print('encoded', df_encoded.columns.tolist())
print('new shape', df.shape, '->', df_encoded.shape)
df_encoded.sample(3)

INFO:init


col @metadata
col @timestamp
col activity_id
col beat
col computer_name
col event_data
col event_id
col host
col keywords
col level
col log_name
col message
col opcode
col process_id
col provider_guid
col record_number
col source_name
col task
col thread_id
col type
col user
col user_data
col version
col event_data_AccessList
col event_data_AccessMask
col event_data_AccessReason
col event_data_Application
col event_data_AuthenticationPackageName
col event_data_CallTrace
col event_data_CommandLine
col event_data_Company
col event_data_ContextInfo
col event_data_CreationUtcTime
col event_data_CurrentDirectory
col event_data_Description
col event_data_DestAddress
col event_data_DestPort
col event_data_DestinationHostname
col event_data_DestinationIp
col event_data_DestinationIsIpv6
col event_data_DestinationPort
col event_data_DestinationPortName
col event_data_Details
col event_data_Device
col event_data_Direction
col event_data_DisabledPrivilegeList
col event_data_ElevatedToken
col even

id  @timestamp  \
1785  1785    0.553282   
2548  2548    0.967179   
1688  1688    0.529124   

      onehot_beat_{'hostname': 'WECserver', 'version': '6.7.0', 'name': 'WECserver'}  \
1785                                                0.0                                
2548                                                1.0                                
1688                                                0.0                                

      onehot_beat_{'name': 'WECserver', 'hostname': 'WECserver', 'version': '6.7.0'}  \
1785                                                1.0                                
2548                                                0.0                                
1688                                                1.0                                

      onehot_beat_{'version': '6.7.0', 'name': 'WECserver', 'hostname': 'WECserver'}  \
1785                                                0.0                                
2548                                                0.0                                
1688                                                0.0                                

      onehot_computer_name_HFDC01.shire.com  \
1785                                    0.0   
2548                                    0.0   
1688                                    0.0   

      onehot_computer_name_HR001.shire.com  \
1785                                   1.0   
2548                                   1.0   
1688                                   1.0   

      onehot_computer_name_IT001.shire.com  event_id  \
1785                                   0.0  0.000168   
2548                                   0.0  0.096350   
1688                                   0.0  0.014934   

      onehot_keywords_['Audit Failure']  onehot_keywords_['Audit Success']  \
1785                                0.0                                0.0   
2548                                0.0                                1.0   
1688                                0.0                                0.0   

      onehot_keywords_['Classic']  onehot_keywords_nan  onehot_level_Error  \
1785                          0.0                  1.0                 0.0   
2548                          0.0                  0.0                 0.0   
1688                          1.0                  0.0                 0.0   

      onehot_level_Information  onehot_level_Verbose  onehot_level_Warning  \
1785                       1.0                   0.0                   0.0   
2548                       1.0                   0.0                   0.0   
1688                       1.0                   0.0                   0.0   

      onehot_log_name_Microsoft-Windows-PowerShell/Operational  \
1785                                                0.0          
2548                                                0.0          
1688                                                0.0          

      onehot_log_name_Microsoft-Windows-Sysmon/Operational  \
1785                                                1.0      
2548                                                0.0      
1688                                                0.0      

      onehot_log_name_Microsoft-Windows-WMI-Activity/Operational  \
1785                                                0.0            
2548                                                0.0            
1688                                                0.0            

      onehot_log_name_Security  onehot_log_name_Windows PowerShell  \
1785                       0.0                                 0.0   
2548                       1.0                                 0.0   
1688                       0.0                                 1.0   

      onehot_opcode_Info  onehot_opcode_On create calls  \
1785                 1.0                            0.0   
2548                 1.0                            0.0   
1688                 1.0                            0.0   

 

### UMAP + K-nn helper

In [10]:
def knn_adjacency_to_edgelist(a_gdf, w_gdf, index_to_id_gdf, drop_far_neighbors=True, drop_far_neighbors_std=1):
    #get rid of spurious children seen in small clusters:
    #assuming n is "good", threshold as nth-furthest's (avgd dist + 1 stddev)
    dist_thresh = None
    scalar = 1.0 if drop_far_neighbors else 0.0 #hack to ensure not dropping
    if drop_far_neighbors:
        dist_thresh = w_gdf[ len(w_gdf.columns) - 1 ].mean() + w_gdf[ len(w_gdf.columns) - 1 ].std() * drop_far_neighbors_std
        print('dropping neighbors > %s away (%s neighbor dist mean + std' % (dist_thresh, len(a_gdf.columns)) )
    else:
        dist_thresh = w_gdf.max().max() #nop, TODO just skip the filter
        print('not dropping neighbors')
            
    edges_gdf = cudf.concat([        
        a_gdf[ [n] ].assign(knn_d=w_gdf[n], knn_n=n) #[[ n, 'knn_d' ]] \
            [ w_gdf[n] * scalar <= dist_thresh ]\
            .reset_index().rename(columns={'index': 'from_idx', n: 'to_idx'}) #[[ 'from_index', 'to_index', 'kn__d', 'knn_n']] \
            .merge(index_to_id_gdf.rename(columns={'id': 'from'}),
                   left_on='from_idx',
                   right_on='index').drop(columns=['index', 'from_idx'])\
            .merge(index_to_id_gdf.rename(columns={'id': 'to'}),
                   left_on='to_idx',
                   right_on='index').drop(columns=['index', 'to_idx']) #[[ 'from', 'to', 'knn_d', 'knn_n']]
        for n in a_gdf.columns
    ], ignore_index=True)
    print('%s edges (dropped %s%% edges)' % (
        len(edges_gdf), 
        round(100.0 * (1.0 - (len(edges_gdf) / (len(a_gdf) * len(a_gdf.columns)))))))

    edges_no_self_gdf = edges_gdf[ edges_gdf['from'] != edges_gdf['to'] ]
    print('%s edges (dropped %s%% edges)' % (
        len(edges_no_self_gdf), 
        round(100.0 * (1.0 - (len(edges_no_self_gdf) / (len(a_gdf) * len(a_gdf.columns - 1)))))))
    return edges_no_self_gdf

def nn(projection_gdf, n_neighbors=20, projection_cols=['x','y']):
    nn_model = cuml.NearestNeighbors(n_neighbors=n_neighbors)
    nn_model.fit(projection_gdf[projection_cols])
    (weights_gdf, adjacency_gdf) = nn_model.kneighbors(projection_gdf[projection_cols])
    return (weights_gdf, adjacency_gdf)


def nn_edges(projection_gdf, index_to_id, n_neighbors=20, drop_far_neighbors=True, drop_far_neighbors_std=1, projection_cols=['x','y']):
    print('computing nn (%s)' % n_neighbors)
    (weights_gdf, adjacency_gdf) = nn(projection_gdf, n_neighbors, projection_cols)
    edges_gdf = knn_adjacency_to_edgelist(adjacency_gdf, weights_gdf, index_to_id, drop_far_neighbors, drop_far_neighbors_std)
    out = edges_gdf.assign(type='knn_%s' % n_neighbors)
    out['weight'] = edges_gdf['knn_d']
    return out

##FIXME buggy some reason
def with_nn_edges(g, n_neighbors=20, drop_far_neighbors=True, drop_far_neighbors_std=1):
    print('with_nn_edges')
    xy_gdf = cudf.from_pandas(g._nodes[['x', 'y']])
    index_to_id_gdf = cudf.from_pandas(g._nodes[[g._node]]).reset_index()\
        .rename(columns={g._node: 'id'})
    edges_gdf = nn_edges(xy_gdf, index_to_id_gdf, n_neighbors, drop_far_neighbors, drop_far_neighbors_std, ['x', 'y'])
    print('got %s edges' % len(edges_gdf))
    edges_pdf = edges_gdf.to_pandas().rename(columns={
        'from': g._source,
        'to': g._destination
    })
    return g.edges(pd.concat([ g._edges, edges_pdf], ignore_index=True, sort=False))

def umap_plot(pdf, id_col=None, fit_opts={}, umap_opts={}, nn_opts=None):
    
    if id_col is None:
        pdf = pdf.reset_index()
        id_col = 'index'
    
    logger.debug('to gdf')
    gdf = cudf.from_pandas(pdf)

    reducer = cuml.UMAP(**umap_opts)
    
    logger.debug('fit')
    embedding = reducer.fit_transform(gdf.drop(columns=[id_col]),
                                      **fit_opts)
    
    nodes_gdf = gdf.assign(x=embedding[0], y=embedding[1])

    logger.debug('graphistry')
    g = graphistry.nodes(nodes_gdf.to_pandas())\
        .bind(node=id_col)\
        .bind(source='s', destination='d').edges(pd.DataFrame({
            's': [ gdf[id_col].head(1).tolist()[0] ],
            'd': [ gdf[id_col].head(1).tolist()[0] ]
        })).settings(url_params={'play': 0, 'strongGravity': True})
    
    logger.debug('nn')
    if not (nn_opts is None):
        g = with_nn_edges(**{'g': g, **nn_opts})
    
    return g

### UMAP

* Place similar events next to one another using x/y coordinates
* Explicitly connect them with edges

In [150]:
%%time

g = umap_plot(
    df_encoded, 
    id_col='id', 
    umap_opts={'n_neighbors': 25, 'repulsion_strength': 5, 'n_epochs': 5000},
    nn_opts={'drop_far_neighbors_std': 0.5} #set to None for no edges
)

g = g.nodes(
    g._nodes[['id', 'x', 'y']]\
        .merge(df.reset_index().rename(columns={'index': 'id'}, copy=False),
               on='id', how='left'))
print(len(g._nodes), len(g._edges))

with_nn_edges
computing nn (20)
dropping neighbors > 0.9144366488486487 away (20 neighbor dist mean + std


INFO:add pending dealloc: cuMemFree_v2 206480 bytes
INFO:add pending dealloc: cuMemFree_v2 412960 bytes
INFO:add pending dealloc: cuMemFree_v2 20648 bytes


51173 edges (dropped 1% edges)
48604 edges (dropped 6% edges)
got 48604 edges
2581 48605


In [291]:
g._nodes.sample(3)

,id,x,y,@metadata,@timestamp,activity_id,beat,computer_name,event_data,event_id,host,keywords,level,log_name,message,opcode,process_id,provider_guid,record_number,source_name,task,thread_id,type,user,user_data,version,event_data_AccessList,event_data_AccessMask,event_data_AccessReason,event_data_Application,event_data_AuthenticationPackageName,event_data_CallTrace,event_data_CommandLine,event_data_Company,event_data_ContextInfo,event_data_CreationUtcTime,event_data_CurrentDirectory,event_data_Description,event_data_DestAddress,event_data_DestPort,event_data_DestinationHostname,event_data_DestinationIp,event_data_DestinationIsIpv6,event_data_DestinationPort,event_data_DestinationPortName,event_data_Details,event_data_Device,event_data_Direction,event_data_DisabledPrivilegeList,event_data_ElevatedToken,...,event_data_SourceHandleId,event_data_SourceHostname,event_data_SourceImage,event_data_SourceIp,event_data_SourceIsIpv6,event_data_SourcePort,event_data_SourcePortName,event_data_SourceProcessGUID,event_data_SourceProcessId,event_data_SourceThreadId,event_data_Status,event_data_SubjectDomainName,event_data_SubjectLogonId,event_data_SubjectUserName,event_data_SubjectUserSid,event_data_TargetDomainName,event_data_TargetFilename,event_data_TargetHandleId,event_data_TargetImage,event_data_TargetInfo,event_data_TargetLinkedLogonId,event_data_TargetLogonGuid,event_data_TargetLogonId,event_data_TargetObject,event_data_TargetOutboundDomainName,event_data_TargetOutboundUserName,event_data_TargetProcessGUID,event_data_TargetProcessId,event_data_TargetServerName,event_data_TargetSid,event_data_TargetUserName,event_data_TargetUserSid,event_data_TerminalSessionId,event_data_TicketEncryptionType,event_data_TicketOptions,event_data_TokenElevationType,event_data_TransactionId,event_data_TransmittedServices,event_data_User,event_data_UtcTime,event_data_VirtualAccount,event_data_WorkstationName,event_data_param1,event_data_param2,event_data_param3,host_name,user_domain,user_identifier,user_name,user_type
1141,1141,39.076866,5.690883,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:43:43.294000+00:00,nan,"{'name': 'WECserver', 'hostname': 'WECserver',...",HR001.shire.com,"{'EventType': 'CreateKey', 'UtcTime': '2019-05...",12,{'name': 'WECserver'},nan,Information,Microsoft-Windows-Sysmon/Operational,Registry object added or deleted:\nRuleName: \...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2238395,Microsoft-Windows-Sysmon,Registry object added or deleted (rule: Regist...,3144.0,wineventlog,"{'name': 'SYSTEM', 'domain': 'NT AUTHORITY', '...",nan,2.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,HKLM\SOFTWARE,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2019-05-18 20:43:43.273,nan,nan,nan,nan,nan,WECserver,NT AUTHORITY,S-1-5-18,SYSTEM,User
552,552,-29.177279,12.377783,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:43:36.878000+00:00,nan,"{'name': 'WECserver', 'hostname': 'WECserver',...",HR001.shire.com,"{'ProcessId': '2156', 'ImageLoaded': 'C:\\Wind...",7,{'name': 'WECserver'},nan,Information,Microsoft-Windows-Sysmon/Operational,Image loaded:\nRuleName: \nUtcTime: 2019-05-18...,Info,2440.0,{5770385f-c22a-43e0-bf4c-06f5698ffbd9},2237969,Microsoft-Windows-Sysmon,Image loaded (rule: ImageLoad),3144.0,wineventlog,"{'identifier': 'S-1-5-18', 'name': 'SYSTEM', '...",nan,3.0,nan,nan,nan,nan,nan,nan,nan,Microsoft Corporation,nan,nan,nan,Microsoft® C Runtime Library,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2019-05-18 20:43:36.831,nan,nan,nan,nan,nan,WECserver,NT AUTHORITY,S-1-5-18,SYSTEM,User
2005,2005,21.979513,-17.859144,"{'beat': 'winlogbeat', 'type': 'doc', 'version...",2019-05-18 20:44:

In [292]:
g._edges.sample(3)

,s,d,knn_d,knn_n,type,weight
34305,69,2579,0.011049,14.0,knn_20,0.011049
11390,1303,1223,0.011049,5.0,knn_20,0.011049
36292,496,1083,0.139317,15.0,knn_20,0.139317


### Visualize

Optionally override some settings

In [151]:
g.plot()

## Create graph with physical edges

Instead of UMAP's fuzzy edges, we can directly map explicit event -> entity edges, such as `(event1) --> (ip) <-- (event2)`

* Defaults to `direct=False` to include events as nodes
* Set `direct=True` to only show entity nodes without events, e.g., `(ip)--[event1]-->(user_name)`

In [294]:
%%time


# not GPU accelerated.. yet
hg = graphistry.hypergraph(
    g._nodes,
    [
                'computer_name', 'user_name', 'user_identifier',
                'event_data_TargetImage', 'event_data_TargetProcessGUID', 'event_data_TargetObject',
                'event_id',
                'event_data_param1', 'event_data_param2', 'event_data_param3',
                'event_data_DestinationIp', 'event_data_SourceIp'
    ],
    direct=False)

edges_df = hg['graph']._edges

print(edges_df.shape)
print(edges_df.sample(3))

hg['graph'].plot( hg['graph']._edges.sample(1000) )

# links 30972
# events 2581
# attrib entities 715
(30972, 167)
                                    event_data_CallTrace  \
5090                                                 nan   
17274  C:\Windows\SYSTEM32\ntdll.dll+9ea54|C:\Windows...   
26043                                                nan   

      event_data_PreAuthType event_data_TargetLogonGuid  \
5090                     nan                        nan   
17274                    nan                        nan   
26043                    nan                        nan   

      event_data_TargetUserSid           event_data_SourceImage  \
5090                       nan                              nan   
17274                      nan  C:\Windows\system32\svchost.exe   
26043                      nan                              nan   

                                  activity_id                   source_name  \
5090   {c93a7728-017f-0000-326e-4aca7f01d501}  Microsoft-Windows-PowerShell   
17274                           

In [295]:
def drop_supernode_edges(g, max_col_links=None):
    
    if max_col_links is None:
        max_col_links = len(g._nodes) / 10
        logger.info('max_col_links: %s nodes -> %s max', len(g._nodes), max_col_links)
    
    #df[['hit_s': True, g._source: 'a]]
    
    s_counts = g._edges[ g._source ].value_counts()
    s_counts = pd.DataFrame(s_counts[ s_counts > max_col_links ])\
        .assign(hit_s=True)\
        .reset_index()[['hit_s', 'index']]\
        .rename(columns={'index': g._source})[['hit_s', g._source]]
    
    #df[['hit_d': True, g._destination: 'a]]
    d_counts = g._edges[ g._destination ].value_counts()
    d_counts = pd.DataFrame(d_counts[ d_counts > max_col_links ])\
        .assign(hit_d=True)\
        .reset_index()[['hit_d', 'index']]\
        .rename(columns={'index': g._destination})[['hit_d', g._destination]]    

    edges2 = g._edges.copy(deep=False)
    edges2 = edges2.merge(s_counts, on=g._source, how='left')
    edges2 = edges2[ ~ edges2['hit_s'].fillna(False) ]
    edges2 = edges2.merge(d_counts, on=g._destination, how='left')
    edges2 = edges2[ ~ edges2['hit_d'].fillna(False) ]
        
    return edges2

def drop_nan_hyper_edges(g):
    edges = g._edges[ ~ g._edges[g._source].str.contains('::nan') ]
    edges = edges[ ~ edges[g._destination].str.contains('::nan') ]
    return edges
    
g2 = hg['graph'].edges(drop_supernode_edges(hg['graph']))
print('dropping supernode edges', hg['graph']._edges.shape, '->', g2._edges.shape)

g2 = g2.edges(drop_nan_hyper_edges(g2))
print('dropping nan edges->', g2._edges.shape)

g2.plot()

INFO:max_col_links: 3296 nodes -> 329.6 max


dropping supernode edges (30972, 167) -> (4436, 169)
dropping nan edges-> (4436, 169)


## Combine graph of linked events with linked entities

Final task: try to merge the two above graphs by creating a new graph based on dataframes `g._nodes`, `g._edges`, `g2._nodes`, and `g2._edges`


## Next steps

Part of the [Python security RAPIDS GPU & graph one-liners](Tutorial_0_Intro.ipynb)

All GPU Python data science tutorials: [RAPIDS Academy github](https://github.com/RAPIDSAcademy/rapidsacademy)